<a href="https://colab.research.google.com/github/edwin-yan/Passion-Fruit-Disease-Detection/blob/andrew/inference_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Look at `Inference` section: 
https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb

And this file:
https://github.com/ultralytics/yolov5/blob/master/detect.py


# SET UP
## Mounting

Note that this assumes that your google drive folder has the same document layout. You must modify if that is not the case.


From your root Google Drive directory assumes there are these folders:
* `JHU/DNN/FinalProject/best.pt` -- weights
* `JHU/DNN/FinalProject/Test_Images` -- test images

with Test images unzipped in those directories



# **TODO:** GLOBAL VARIABLE set to `True` if using colab and `False` if not!

In [1]:
IS_COLAB = True 

# **TODO:** CHANGE `IMG_PATH` if your google drive directory is different

In [2]:
if IS_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  IMG_PATH = '/content/drive/MyDrive/JHU/DNN/FinalProject'
else:
  IMG_PATH= '.'

Mounted at /content/drive


In [3]:
TEST_IMAGES = f'{IMG_PATH}/Test_Images'

In [4]:
WEIGHTS = f'{IMG_PATH}/best.pt'

Start timer

In [5]:
import time

t0 = time.time()

Inference

https://github.com/ultralytics/yolov5#quick-start-examples

In [6]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 9829, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9829 (delta 1), reused 6 (delta 0), pack-reused 9820
Receiving objects: 100% (9829/9829), 10.22 MiB | 27.62 MiB/s, done.
Resolving deltas: 100% (6796/6796), done.
/content/yolov5
     |████████████████████████████████| 596 kB 29.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Perform inference

In [7]:
! python detect.py --source {TEST_IMAGES} --weights {WEIGHTS} --save-txt --save-conf --conf-thres 0 --max-det 20

detect: weights=['/content/drive/MyDrive/JHU/DNN/FinalProject/best.pt'], source=/content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images, imgsz=[640, 640], conf_thres=0.0, iou_thres=0.45, max_det=20, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-45-g042f02f torch 1.9.0+cu111 CPU

Fusing layers... 
Model Summary: 444 layers, 86186872 parameters, 0 gradients, 204.0 GFLOPs
image 1/931 /content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images/ID_0082Y2NI.jpg: 640x640 20 fruit_woodinesss, Done. (3.336s)
image 2/931 /content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images/ID_00DCPV52.jpg: 640x640 2 fruit_woodinesss, 18 fruit_brownspots, Done. (3.303s)
image 3/931 /content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images/ID_038S0ONN.jpg: 640x640

# TODO: Adjust files to Zindi format

.txt files are in the format of:

.txt per image: 

/content/yolov5/runs/detect/exp/labels

In [8]:
category_index = {0: {'id': 0, 'name': 'fruit_healthy'},
                1: {'id': 1, 'name': 'fruit_woodiness'},
                 2: {'id': 3, 'name': 'fruit_brownspot'}}

In [9]:

import os
import pandas as pd
path = '/content/yolov5/runs/detect/exp/labels/'
filelist = os.listdir(path) 
df_write = pd.DataFrame(columns = ['Image_ID', 'class', 'confidence', 'ymin','xmin','ymax','xmax'])

for file in filelist:
  data = pd.read_csv(path+file, sep=" ", header=None)
  data = data.rename(columns={0:'class',1:'x',2:'y',3:'w',4:'h',5:'confidence'})
  data['class'] = data['class'].apply(lambda x: category_index[x]['name'])
  data['ymin'] = (data['y']-data['h']/2)*512
  data['xmin'] = (data['x']-data['w']/2)*512
  data['ymax'] = (data['y']+data['h']/2)*512
  data['xmax'] = (data['x']+data['w']/2)*512
  data['Image_ID'] = file.split('.')[0]

  cols = ['Image_ID','class','confidence','ymin',	'xmin',	'ymax',	'xmax']
  data = data[cols]

  df_write = df_write.append(data.nlargest(4, 'confidence'))


In [10]:
df_write

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
19,ID_CGND1FCK,fruit_woodiness,0.943193,121.999872,172.999680,438.000128,401.999872
18,ID_CGND1FCK,fruit_woodiness,0.784808,218.000128,132.999936,380.999936,206.999808
17,ID_CGND1FCK,fruit_woodiness,0.001231,237.000192,0.000026,440.000000,17.000013
16,ID_CGND1FCK,fruit_woodiness,0.000640,197.999872,142.999808,396.000000,262.999808
19,ID_S6ROOR7U,fruit_woodiness,0.470940,222.000128,184.999680,373.000192,405.999872
...,...,...,...,...,...,...,...
16,ID_RE12SIOR,fruit_healthy,0.000244,152.000000,325.000192,223.000064,398.000128
19,ID_WKO73V6Q,fruit_woodiness,0.974779,155.000064,3.000320,506.000128,382.000128
18,ID_WKO73V6Q,fruit_woodiness,0.015999,-0.000051,335.999744,63.000013,510.999808
17,ID_WKO73V6Q,fruit_woodiness,0.000946,480.000000,157.000192,512.000000,319.000064


In [11]:
df_write.to_csv('/content/drive/MyDrive/JHU/DNN/FinalProject/submission_yolo_unnormalized_top4.csv', encoding='utf-8', index=False)

In [12]:
tend = time.time()
print('Elapsed time: ', (tend-t0)/60, ' minutes')

Elapsed time:  61.926971991856895  minutes
